In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv


In [2]:
# Load data
train_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Prepare data
X_train = train_data.drop(columns=['label'])

# Convert original label data to categorical, i.e.
# label  ->  0  1  2  3  4  5  6  7  8  9
# ---------------------------------------
#     1  ->  0  1  0  0  0  0  0  0  0  0
#     0  ->  1  0  0  0  0  0  0  0  0  0
#     1  ->  0  1  0  0  0  0  0  0  0  0
#     4  ->  0  0  0  0  1  0  0  0  0  0
#     0  ->  1  0  0  0  0  0  0  0  0  0
from tensorflow.python.keras.utils.np_utils import to_categorical
y_train = to_categorical(train_data.label.to_numpy(), dtype='int')
pd.DataFrame(y_train).head()

,0,1,2,3,4,5,6,7,8,9
0,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0


In [4]:
# Craete model
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Reshape
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Reshape((28, 28, 1), input_shape=(28 * 28,)))

model.add(Conv2D(20, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(40, (4, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
# Fit model
model.fit(X_train, y_train, epochs=50, batch_size=32)

Train on 42000 samples
Epoch 1/50
42000/42000 [==============================] - 20s 484us/sample - loss: 0.4949 - accuracy: 0.9232
Epoch 2/50
42000/42000 [==============================] - 20s 466us/sample - loss: 0.0916 - accuracy: 0.9727
Epoch 3/50
42000/42000 [==============================] - 19s 450us/sample - loss: 0.0662 - accuracy: 0.9801
Epoch 4/50
42000/42000 [==============================] - 19s 461us/sample - loss: 0.0531 - accuracy: 0.9846
Epoch 5/50
42000/42000 [==============================] - 19s 464us/sample - loss: 0.0494 - accuracy: 0.9844
Epoch 6/50
42000/42000 [==============================] - 19s 448us/sample - loss: 0.0482 - accuracy: 0.9859
Epoch 7/50
42000/42000 [==============================] - 19s 445us/sample - loss: 0.0407 - accuracy: 0.9879
Epoch 8/50
42000/42000 [==============================] - 22s 516us/sample - loss: 0.0391 - accuracy: 0.9887
Epoch 9/50
42000/42000 [==============================] - 19s 447us/sample - loss: 0.0328 - accuracy: 0.9

In [6]:
# Submit predictions
predictions = model.predict(test_data.values)

output = pd.DataFrame({'ImageId': pd.Series(range(1, len(predictions) + 1)),
                       'Label': predictions.argmax(axis=1)})
output.to_csv('submission.csv', index=False)
print("Submission was successfully saved!")

Submission was successfully saved!
